In [55]:
import numpy as np
import os
import json
import ast
import apgpy
from sklearn.utils.extmath import randomized_svd

In [56]:
os.listdir()

['.Rhistory',
 'parameters_and_descriptions.xlsx',
 '.~lock.parameters_and_descriptions.xlsx#',
 'python data.json',
 'causal_inference_methods_code_corrupted.R',
 'Survival Analysis Simulation Tools.R',
 'corrupted_control_simulations.Rmd',
 'thing.json',
 'reports',
 'Untitled.ipynb',
 '.ipynb_checkpoints',
 'for_compiling_report.R',
 'Survival Data Generation.R',
 'propensity.score.simulations.R']

In [57]:
def std_logistic_function(x):
    return 1 / (1 + np.exp(-x))


def grad_std_logistic_function(x):
    z = np.exp(-x)
    return z / (1 + z)**2


def mod_logistic_function(x, gamma, one_minus_logistic_gamma):
    x = np.clip(x, -gamma, gamma)
    return 1 / (1 + np.exp(-x)) + .5 * (1 + x/gamma) * one_minus_logistic_gamma


def grad_mod_logistic_function(x, gamma, one_minus_logistic_gamma):
    x = np.clip(x, -gamma, gamma)
    z = np.exp(-x)
    return z / (1 + z)**2 + one_minus_logistic_gamma / (2*gamma)


def one_bit_MC_fully_observed(M, link, link_gradient, tau, gamma, max_rank=None,
                              apg_max_iter=100, apg_eps=1e-12,
                              apg_use_restart=True):
    # parameters are the same as in the paper; if `max_rank` is set to None,
    # then exact SVD is used
    m = M.shape[0]
    n = M.shape[1]
    tau_sqrt_mn = tau * np.sqrt(m*n)

    def prox(_A, t):
        _A = _A.reshape(m, n)

        # project so nuclear norm is at most tau*sqrt(m*n)
        if max_rank is None:
            U, S, VT = np.linalg.svd(_A, full_matrices=False)
        else:
            U, S, VT = randomized_svd(_A, max_rank)
        nuclear_norm = np.sum(S)
        if nuclear_norm > tau_sqrt_mn:
            S *= tau_sqrt_mn / nuclear_norm
            _A = np.dot(U * S, VT)

        # clip matrix entries with absolute value greater than gamma
        mask = np.abs(_A) > gamma
        if mask.sum() > 0:
            _A[mask] = np.sign(_A[mask]) * gamma

        return _A.flatten()

    M_one_mask = (M == 1)
    M_zero_mask = (M == 0)
    def grad(_A):
        _A = _A.reshape(m, n)

        grad = np.zeros((m, n))
        grad[M_one_mask] = -link_gradient(_A[M_one_mask])/link(_A[M_one_mask])
        grad[M_zero_mask] = \
            link_gradient(_A[M_zero_mask])/(1 - link(_A[M_zero_mask]))

        return grad.flatten()

    A_hat = apgpy.solve(grad, prox, np.zeros(m*n),
                        max_iters=apg_max_iter,
                        eps=apg_eps,
                        use_gra=True,
                        use_restart=apg_use_restart,
                        quiet=True)
    P_hat = link(A_hat.reshape(m, n))
    return P_hat


def one_bit_MC_mod_fully_observed(M, link, link_gradient, tau, gamma,
                                  max_rank=None, apg_max_iter=100,
                                  apg_eps=1e-12, apg_use_restart=True,
                                  phi=None):
    # parameters are the same as in the paper; if `max_rank` is set to None,
    # then exact SVD is used
    m = M.shape[0]
    n = M.shape[1]
    tau_sqrt_mn = tau * np.sqrt(m*n)
    M_zero_mask = (M == 0)
    if phi is None:
        phi = .95 * gamma

    def prox(_A, t):
        _A = _A.reshape(m, n)

        # project so nuclear norm is at most tau*sqrt(m*n)
        if max_rank is None:
            U, S, VT = np.linalg.svd(_A, full_matrices=False)
        else:
            U, S, VT = randomized_svd(_A, max_rank)
        nuclear_norm = np.sum(S)
        if nuclear_norm > tau_sqrt_mn:
            S *= tau_sqrt_mn / nuclear_norm
            _A = np.dot(U * S, VT)

        # clip matrix entries with absolute value greater than gamma
        mask = np.abs(_A) > gamma
        if mask.sum() > 0:
            _A[mask] = np.sign(_A[mask]) * gamma

        mask = _A[M_zero_mask] > phi
        if mask.sum() > 0:
            _A[M_zero_mask][mask] = phi

        return _A.flatten()

    M_one_mask = (M == 1)
    def grad(_A):
        _A = _A.reshape(m, n)

        grad = np.zeros((m, n))
        grad[M_one_mask] = -link_gradient(_A[M_one_mask])/ \
            link(np.maximum(_A[M_one_mask], -gamma))
        grad[M_zero_mask] = \
            link_gradient(_A[M_zero_mask])/ \
            (1 - link(np.minimum(_A[M_zero_mask], phi)))

        return grad.flatten()

    A_hat = apgpy.solve(grad, prox, np.zeros(m*n),
                        max_iters=apg_max_iter,
                        eps=apg_eps,
                        use_gra=True,
                        use_restart=apg_use_restart,
                        quiet=True)
    P_hat = link(A_hat.reshape(m, n))
    return P_hat


def weighted_softimpute(X, M, W, lmbda, max_rank=None,
                        min_value=None, max_value=None,
                        apg_max_iter=100, apg_eps=1e-6,
                        apg_use_restart=True):
    # if `max_rank` is set to None, then exact SVD is used
    m = X.shape[0]
    n = X.shape[1]

    def prox(Z, t):
        Z = Z.reshape(m, n)

        # singular value shrinkage
        if max_rank is None:
            U, S, VT = np.linalg.svd(Z, full_matrices=False)
        else:
            U, S, VT = randomized_svd(Z, max_rank)
        S = np.maximum(S - lmbda*t, 0)
        Z = np.dot(U * S, VT)

        # clip values
        if min_value is not None:
            mask = Z < min_value
            if mask.sum() > 0:
                Z[mask] = min_value
        if max_value is not None:
            mask = Z > max_value
            if mask.sum() > 0:
                Z[mask] = max_value

        return Z.flatten()

    M_one_mask = (M == 1)
    masked_weights = W[M_one_mask]
    masked_X = X[M_one_mask]
    def grad(Z):
        grad = np.zeros((m, n))
        grad[M_one_mask] = (Z.reshape(m, n)[M_one_mask] - masked_X) * masked_weights
        return grad.flatten()

    X_hat = apgpy.solve(grad, prox, np.zeros(m*n),
                        max_iters=apg_max_iter,
                        eps=apg_eps,
                        use_gra=True,
                        use_restart=apg_use_restart,
                        quiet=True).reshape((m, n))
    return X_hat

In [43]:
file_path = "python data.json"

with open(file_path) as f:
    data = json.loads(f.read())

dataInDict = ast.literal_eval(data[0])

In [51]:
yArray = np.array(dataInDict['Y'])

W = np.array(dataInDict['W'])

In [62]:
thingus = one_bit_MC_fully_observed(M=W, link=std_logistic_function, link_gradient=grad_std_logistic_function, 
                                    tau=9, gamma=100, max_rank=None,
                              apg_max_iter=100, apg_eps=1e-12,
                              apg_use_restart=True)

In [63]:
thingus

array([[0.00221071, 0.00221071, 0.00221071, ..., 0.00221071, 0.00221071,
        0.00221071],
       [0.00221071, 0.00221071, 0.00221071, ..., 0.00221071, 0.00221071,
        0.00221071],
       [0.00221071, 0.00221071, 0.00221071, ..., 0.00221071, 0.00221071,
        0.00221071],
       ...,
       [0.00221071, 0.00221071, 0.00221071, ..., 0.00221071, 0.00221071,
        0.00221071],
       [0.00221071, 0.00221071, 0.00221071, ..., 0.00221071, 0.00221071,
        0.00221071],
       [0.00221071, 0.00221071, 0.00221071, ..., 0.00221071, 0.00221071,
        0.00221071]])